This cleaning file results in one combined file (poverty.csv) containing data on poverty.    

Data source: https://www.census.gov/programs-surveys/saipe/data/datasets.html

In [2]:
import os
import sys
import requests
import pandas as pd


current_dir =  os.getcwd()

grandparent_dir = os.path.dirname(os.path.dirname(current_dir))
sys.path.insert(0, grandparent_dir)

In [28]:

dataframes = []

for year in range(1998, 2021 + 1):
 
    api_url = f"https://api.census.gov/data/timeseries/poverty/saipe?get=YEAR,GEOID,SAEMHI_PT,SAEPOV0_17_PT,SAEPOVRT0_17_PT,SAEPOVALL_PT,SAEPOVRTALL_PT&for=county:*&in=state:*&time={year}"
    

    response = requests.get(api_url)
    
   
    if response.status_code == 200:
       
        data = response.json()
        columns = data[0]
        data = data[1:]
        df = pd.DataFrame(data, columns=columns)
        dataframes.append(df)
        print(f"DataFrame for year {year} created successfully.")

poverty_raw = pd.concat(dataframes, ignore_index=True)

DataFrame for year 1998 created successfully.
DataFrame for year 1999 created successfully.
DataFrame for year 2000 created successfully.
DataFrame for year 2001 created successfully.
DataFrame for year 2002 created successfully.
DataFrame for year 2003 created successfully.
DataFrame for year 2004 created successfully.
DataFrame for year 2005 created successfully.
DataFrame for year 2006 created successfully.
DataFrame for year 2007 created successfully.
DataFrame for year 2008 created successfully.
DataFrame for year 2009 created successfully.
DataFrame for year 2010 created successfully.
DataFrame for year 2011 created successfully.
DataFrame for year 2012 created successfully.
DataFrame for year 2013 created successfully.
DataFrame for year 2014 created successfully.
DataFrame for year 2015 created successfully.
DataFrame for year 2016 created successfully.
DataFrame for year 2017 created successfully.
DataFrame for year 2018 created successfully.
DataFrame for year 2019 created su

In [38]:
poverty_raw.rename(columns={'YEAR': 'Year', 'GEOID': 'GeoFIPS', 'SAEMHI_PT': 'medianHouseholdIncome',
'SAEPOV0_17_PT': 'povertyUnder18', 'SAEPOVRTALL_PT':'povertyAllpr', 'SAEPOVALL_PT':'povertyAll',
'SAEPOVRT0_17_PT': 'povertyUnder18pr'
    }, inplace=True)

columns_to_keep = ['GeoFIPS', 'Year', 'povertyAll', 'povertyAllpr', 
        'povertyUnder18', 'povertyUnder18pr', 'medianHouseholdIncome']
poverty_raw = poverty_raw.loc[:, columns_to_keep]

In [39]:
poverty_raw

,GeoFIPS,Year,povertyAll,povertyAllpr,povertyUnder18,povertyUnder18pr,medianHouseholdIncome
0,01001,1998,4901,11.3,2149,17.7,38260
1,01003,1998,15089,11.0,5688,16.8,37162
2,01005,1998,5795,22.7,2183,31.3,27603
3,01007,1998,3313,16.9,1279,23.7,29692
4,01009,1998,6176,12.9,2313,19.8,35623
...,...,...,...,...,...,...,...
75399,56037,2021,3761,9.2,990,9.7,74677
75400,56039,2021,1392,5.9,223,5.5,102709
75401,56041,2021,1917,9.4,635,11.2,70162
75402,56043,2021,773,10.2,195,12.5,62176


In [27]:
poverty_raw.to_csv(f"{grandparent_dir}/data/raw/poverty.csv", index=False)


NameError: name 'poverty_raw' is not defined

In [28]:
poverty = pd.read_csv(f"{grandparent_dir}/data/raw/poverty.csv")
from cities.utils.data_grabber import DataGrabber

In [8]:
data = DataGrabber()
data.get_features_wide(["gdp"])
gdp = data.wide["gdp"]

import numpy as np

In [29]:
poverty

,GeoFIPS,GeoName,Year,povertyAll,povertyAllpr,povertyUnder18,povertyUnder18pr,medianHouseholdIncome
0,1001,"Autauga, AL",1998,4901.0,11.3,2149.0,17.7,38260.0
1,1001,"Autauga, AL",1999,4991.0,11.4,2115.0,16.9,39702.0
2,1001,"Autauga, AL",2000,4681.0,10.5,1749.0,14.1,42463.0
3,1001,"Autauga, AL",2001,4904.0,10.8,1823.0,14.7,42183.0
4,1001,"Autauga, AL",2002,4795.0,10.3,1816.0,14.4,42841.0
...,...,...,...,...,...,...,...,...
73737,56045,"Weston, WY",2017,699.0,10.6,198.0,14.0,56214.0
73738,56045,"Weston, WY",2018,777.0,11.8,227.0,16.1,54319.0
73739,56045,"Weston, WY",2019,692.0,10.5,189.0,13.8,59410.0
73740,56045,"Weston, WY",2020,679.0,10.6,168.0,12.6,58901.0


In [14]:
common_fips = np.intersect1d(gdp["GeoFIPS"].unique(), poverty["GeoFIPS"].unique())

poverty = poverty[poverty["GeoFIPS"].isin(common_fips)]

poverty = poverty.merge(gdp[["GeoFIPS", "GeoName"]], on="GeoFIPS", how="left")

poverty = poverty.sort_values(by=["GeoFIPS", "GeoName"])

In [16]:
desired_order = ['GeoFIPS', 'GeoName', 'Year', 'povertyAll', 'povertyAllpr',
                  'povertyUnder18', 'povertyUnder18pr', 'medianHouseholdIncome' ]

poverty = poverty[desired_order]


In [23]:
poverty.to_csv(f"{grandparent_dir}/data/raw/poverty.csv", index=False)
